### Project 02: Create a GUI Notebook Program

Young Lee
3/30/2025

Project 2 will adapt the procedural code we have been working on in INST326_SimpleGUI_Note_Form_IO.ipynb to create an OOP version of the program using three classes.

    A Notebook or MainWindow class
    A Form or TopWindow class
    A Note class

The MainWindow class is a subclass of Tkinter’s tk.Tk class and thus inherits its attributes and methods, while allowing us to customize the new window objects to our needs. These new window objects will represent “notebooks” or collections of notes, and allow us to work with those notes. (I have named it MainWindow because this class definition could be used to create any kind of main window in Tkinter. We are using it to represent notebooks in this application, but you might use it for other purposes in onther applications.)


The TopWindow class creates a new top window in Tkinter, which is essentially a form for entering the title, text, links, and tags for our note. When we submit the note, this “form” object has a method that creates the note’s metadata and a new Note object. That note object is appended to the list of all notes, which is an attribute of the notebook (MainWindow) class.
The Note class creates note objects that contain the  title, text, links, tags, and metadata for each note.

For Project 02 you will:  

    1. Create one notebook or MainWindow object  
    2. Create several (at least 3) ‘real’ notes for your final submission. By ‘real’ I mean actual notes about python that are useful to you. Print them in the cell at the bottom of the notebook.
    3. Save those notes to a single .txt, .csv, or .json file (your choice of format).  
    4. Retrieve those notes and 
    5. Display representations of them as either labels or buttons in the  main window (remember how we displayed cards in project 01). These representations are not whole notes. Rather, they are object representations of the notes.  
    6. When they are clicked the whole note pops up in a new window - either the form window or a ‘read-only’ version of the form window.



#### Complete your code in the cell below

The code cell below contains the imports you will need; the top level structure for the three classes to get you started; and the execution code at the bottom. 

In [11]:
# imports
import tkinter as tk
from tkinter import ttk, filedialog
import datetime  # module for working with dates and times
import json
import os

# The MainWindow class creates a custom GUI window based on the tkinter window (tk.Tk)
# It has an __init__() method, and three additional methods (new_note(), open_notebook(), and save_notebook())
# These methods correspond to new, open, and save buttons in the window.
# The new_note method calls the NoteForm class to create a new note form top level window.

class MainWindow(tk.Tk):
    def __init__(self):  # initialize the main window
        super().__init__()  # initialize it as a tkinter window

        self.geometry("600x400")  # set the default window size
        self.title('Notebook')  # set the default window title
        self.notebook = []  # initialize an attribute named 'notebook' as an empty list
        self.notes = [
            
        ]  # list to store note objects
        self.notes_widgets = []  # list to store note widgets

        # add additional lines to the __init__() function
        new_button = ttk.Button(self, text="New", command=self.new_note)
        new_button.pack(side=tk.LEFT, padx=5)

        open_button = ttk.Button(self, text="Open", command=self.open_notebook)
        open_button.pack(side=tk.LEFT, padx=5)

        save_button = ttk.Button(self, text="Save", command=self.save_notebook)
        save_button.pack(side=tk.LEFT, padx=5)

        # Add a frame to hold the note widgets
        self.notes_frame = ttk.Frame(self)
        self.notes_frame.pack(fill=tk.BOTH, expand=True)

    def new_note(self):
        note_window = NoteForm(self, self.notebook, self.notes)
        return None

    def open_notebook(self):
        filepath = filedialog.askopenfilename(initialdir="C:\\Users\\sdemp\\Documents\\GitHub\\Courses\\INST326\\test_files",
                                             filetypes=[("text files", "*.txt"), 
                                                        ("csv files", ".csv"),
                                             ("all files", "*.*")])
        if filepath:
            filename = os.path.basename(filepath)
            with open(filepath, "r") as file:
                lines = file.readlines()
                if len(lines) >= 2:
                    note_title, note_text = lines[:2]
                    note_title = note_title.strip()
                    note_text = note_text.strip()
                    self.notebook.append({
                        "title": filename,
                        "text": note_text,
                        "date_created": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    })
                    print("Notebook opened successfully.")
                    print(filename)
                    self.display_notes()
                else:
                    print("Invalid notebook file format.")


    def save_notebook(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".json", filetypes=[("JSON files", "*.json")])
        if file_path:
            with open(file_path, "w") as file:
                json.dump(self.notebook, file, indent=4)
                print("Notebook saved successfully.")

    def display_notes(self):
        # Clear previous note widgets
        for widget in self.notes_widgets:
            widget.destroy()
        self.notes_widgets.clear()

        # Create new note widgets
        for note in self.notebook:
            note_text = f"Title: {note['title']}\nText: {note['text']}\nDate Created: {note['date_created']}"
            note_button = ttk.Button(self.notes_frame, text=note['title'], command=lambda n=note_text: self.show_note_detail(n))
            note_button.pack(padx=5, pady=5, fill=tk.BOTH)
            self.notes_widgets.append(note_button)

    def show_note_detail(self, note_text):
        detail_window = tk.Toplevel(self)
        detail_window.title("Note Detail")
        detail_label = ttk.Label(detail_window, text=note_text)
        detail_label.pack(padx=10, pady=10)

# the NoteForm() class creates a Toplevel window that is a note form containing fields for
# data entry for title, text, link, and tags. It also calculates a meta field with date, time, and timezone
# the Noteform class has an __init__() method, and a submit() method that is called by a submit button
# the class may contain additional methods to perform tasks like calculating the metadata, for example
# the submit method calls the MakeNote class that transforms the the entered data into a new note object.

class NoteForm(tk.Toplevel):

    def __init__(self, master, notebook, notes):  # initialize the new object
        super().__init__(master)  # initialize it as a toplevel window

        # add additional lines to set the new window's attributes and default parameters
        self.title('New Note')
        self.geometry("400x300")
        self.notebook = notebook
        self.notes = notes

        self.note_title = tk.StringVar()
        self.note_text = tk.StringVar()
        self.note_link = tk.StringVar()
        self.note_tags = tk.StringVar()

        title_label = ttk.Label(self, text="Title:")
        title_label.pack()

        title_entry = ttk.Entry(self, textvariable=self.note_title)
        title_entry.pack()

        text_label = ttk.Label(self, text="Text:")
        text_label.pack()

        text_entry = ttk.Entry(self, textvariable=self.note_text)
        text_entry.pack()

        link_label = ttk.Label(self, text="Link:")
        link_label.pack()

        link_entry = ttk.Entry(self, textvariable=self.note_link)
        link_entry.pack()

        tags_label = ttk.Label(self, text="Tags:")
        tags_label.pack()

        tags_entry = ttk.Entry(self, textvariable=self.note_tags)
        tags_entry.pack()

        submit_button = ttk.Button(self, text="Submit", command=self.submit)
        submit_button.pack()

    def submit(self):
        note_dict = {
            "title": self.note_title.get(),
            "text": self.note_text.get(),
            "link": self.note_link.get(),
            "tags": self.note_tags.get(),
            "date_created": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }
        self.notebook.append(note_dict)
        self.master.display_notes()  # Update the displayed notes
        self.destroy()

# The MakeNote class takes a dictionary containing the data entered into the form window,
# and transforms it into a new note object.
# At present the note objects have attributes but no methods.

class MakeNote():
    def __init__(self, note_dict):
        # add lines to the init method
        self.title = note_dict["title"]
        self.text = note_dict["text"]
        self.link = note_dict["link"]
        self.tags = note_dict["tags"]
        self.date_created = note_dict["date_created"]

# main execution

if __name__ == '__main__':
    main_window = MainWindow()  # this creates a notebook / main window called main_window.
    main_window.mainloop()


#### Print your three notes below

In [6]:
# print your notes here
print("Notes:", main_window.notebook)

Notes: [{'title': 'Versatility and Application', 'text': "Python's versatility makes it a valuable tool for college students across various disciplines. From data analysis in STEM fields to web development and automation in business or humanities, Python's wide range of libraries and frameworks cater to diverse needs. Learning Python equips students with a practical skillset applicable in academia and beyond.", 'link': 'link', 'tags': 'python, versatility, application', 'date_created': '2024-03-31 17:49:30'}, {'title': 'Community and Resources', 'text': 'Python boasts a vibrant community with extensive documentation, tutorials, and forums. College students can leverage this wealth of resources to deepen their understanding, troubleshoot issues, and collaborate on projects. Platforms like Stack Overflow, GitHub, and Python-specific forums offer invaluable support for beginners and advanced learners alike.', 'link': 'link', 'tags': 'python, community, resources', 'date_created': '2024-03